## Web Crawling
- 웹 페이지에서 데이터를 수집하는 방법에 대해서 학습

### 웹크롤링 방법

#### 웹페이지의 종류
- 정적인 페이지 : 웹 브라우져에 화면이 한번 뜨면 이벤트에 의한 화면의 변경이 없는 페이지 
- 동적인 페이지 : 웹 브라우져에 화면이 뜨고 이벤트가 발생하면 서버에서 데이터를 가져와 화면을 변경하는 페이지

#### requests 이용
- 받아오는 문자열에 따라 두가지 방법으로 구분
    - json 문자열로 받아서 파싱하는 방법 : 주로 동적 페이지 크롤링할때 사용 
    - html 문자열로 받아서 파싱하는 방법 : 주로 정적 페이지 크롤링할때 사용
        
#### selenium 이용
- 브라우져를 직접 열어서 데이터를 받는 방법

#### 크롤링 방법에 따른 속도
- requests json > requests html > selenium

html은 태그 = (상대적)과대포장이라서 포장 뜯는데 json보다 오래걸려서 속도가 느림
셀리니움은 당연히 크롬브라우저 열고 이러느라 훨씬 오랜 시간이 걸림

### Crawling Naver Stock Datas
- 네이버 증권 사이트에서 주가 데이터 수집
- 수집할 데이터 : 일별 kospi, kosdaq 주가, 일별 환율(exchange rate) 데이터
- 데이터 수집 절차
    - 웹서비스 분석 : url
    - 서버에 데이터 요청 : request(url) > response : json(str)
    - 서버에서 받은 데이터 파싱(데이터 형태를 변경) : json(str) > list, dict > DataFrame

In [29]:
import requests
import pandas as pd

#### 1. 웹서비스 분석 : url
- pc 웹페이지가 복잡하면 mobile 웹페이지에서 수집

In [31]:
page_size,page = 60, 1
url = f'https://m.stock.naver.com/api/index/KOSPI/price?pageSize={page_size}&page={page}'
url

'https://m.stock.naver.com/api/index/KOSPI/price?pageSize=60&page=1'

#### 2. 서버에 데이터 요청 : request(url) > response : json(str)
- response의 status code가 200이 나오는지 확인
- 403이나 500이 나오면 request가 잘못되거나 web server에서 수집이 안되도록 설정이 된것임
    - header 설정 또는 selenium 사용
- 200이 나오더라도 response 안에 있는 내용을 확인 > 확인하는 방법 : response.text

In [33]:
response = requests.get(url)
response

<Response [200]>

In [34]:
response.text[:2]

'[{'

In [35]:
type(response)

requests.models.Response

In [36]:
dir(response) #변수, 함수 출력

['__attrs__',
 '__bool__',
 '__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__enter__',
 '__eq__',
 '__exit__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getstate__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__iter__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__nonzero__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__setstate__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_content',
 '_content_consumed',
 '_next',
 'apparent_encoding',
 'close',
 'connection',
 'content',
 'cookies',
 'elapsed',
 'encoding',
 'headers',
 'history',
 'is_permanent_redirect',
 'is_redirect',
 'iter_content',
 'iter_lines',
 'json',
 'links',
 'next',
 'ok',
 'raise_for_status',
 'raw',
 'reason',
 'request',
 'status_code',
 'text',
 'url']

In [37]:
type(response.text) , response.text[:100]

(str,
 '[{"localTradedAt":"2024-09-20","closePrice":"2,593.37","compareToPreviousClosePrice":"12.57","compar')

#### 3. 서버에서 받은 데이터 파싱(데이터 형태를 변경) : json(str) > list, dict > DataFrame

In [39]:
data =response.json() #문자열 데이터가 리스트, 딕셔너리의 형태로 바뀜

type(data) , data[:2]

(list,
 [{'localTradedAt': '2024-09-20',
   'closePrice': '2,593.37',
   'compareToPreviousClosePrice': '12.57',
   'compareToPreviousPrice': {'code': '2', 'text': '상승', 'name': 'RISING'},
   'fluctuationsRatio': '0.49',
   'openPrice': '2,603.83',
   'highPrice': '2,619.55',
   'lowPrice': '2,591.40'},
  {'localTradedAt': '2024-09-19',
   'closePrice': '2,580.80',
   'compareToPreviousClosePrice': '5.39',
   'compareToPreviousPrice': {'code': '2', 'text': '상승', 'name': 'RISING'},
   'fluctuationsRatio': '0.21',
   'openPrice': '2,594.67',
   'highPrice': '2,598.68',
   'lowPrice': '2,550.09'}])

In [40]:
df=pd.DataFrame(data)

df = df[['localTradedAt' , 'closePrice']]
df.tail(2)

,localTradedAt,closePrice
58,2024-06-26,"2,792.05"
59,2024-06-25,"2,774.39"


#### 4. 함수로 만들기

In [42]:
def stock_price(page = 1 ,page_size = 60):
    
    
    url = f'https://m.stock.naver.com/api/index/KOSPI/price?pageSize={page_size}&page={page}'
    response = requests.get(url)
    data =response.json()

    return pd.DataFrame(data)[['localTradedAt','closePrice']] # 기존 과정을 함수화

In [43]:
stock_price(page=3,page_size=5)

,localTradedAt,closePrice
0,2024-09-03,"2,664.63"
1,2024-09-02,"2,681.00"
2,2024-08-30,"2,674.31"
3,2024-08-29,"2,662.28"
4,2024-08-28,"2,689.83"


In [44]:
kospi_df = stock_price()
kospi_df.tail(2)

,localTradedAt,closePrice
58,2024-06-26,"2,792.05"
59,2024-06-25,"2,774.39"


In [45]:
kosdaq_df = stock_price("KOSDAQ")
kosdaq_df.tail(2)

JSONDecodeError: Expecting value: line 1 column 1 (char 0)

#### 5. 원달러 환율 데이터 수집 : 실습

In [ ]:
# 코스닥 데이터 수집
# 원달러 환율 데이터 수집 https://m.stock.naver.com/marketindex/exchange/FX_USDKRW 환율데이터

def exchage_rate(code="FX_USDKRW", page=1, page_size=60):
    url = f'https://m.stock.naver.com/front-api/marketIndex/prices?\
page={page}&category=exchange&reutersCode={code}&pageSize={page_size}'
    response = requests.get(url)
    columns = ["localTradedAt", "closePrice"]
    data = response.json()['result']
    return pd.DataFrame(data)[columns]

In [ ]:
usd_df = exchage_rate()
usd_df.tail(2)

#### 6. 시각화

In [ ]:
%matplotlib inline
%config InlineBackend.figure_formats = {'png', 'retina'}

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
# 데이터 수집
page_size = 60
kospi_df = stock_price("KOSPI", page_size=page_size)
kosdaq_df = stock_price("KOSDAQ", page_size=page_size)
usd_df = exchage_rate("FX_USDKRW", page_size=page_size)

In [ ]:
# 데이터 전처리 1 : 데이터 타입 변경
print(kospi_df.dtypes)
kospi_df["kospi"] = kospi_df["closePrice"].apply(
    lambda data: float(data.replace(",", ""))
)
kospi_df = kospi_df.drop(columns=["closePrice"])
print(kospi_df.dtypes)

In [ ]:
kosdaq_df["kosdaq"] = kosdaq_df["closePrice"].apply(
    lambda data: float(data.replace(",", ""))
)
usd_df["usd"] = usd_df["closePrice"].apply(
    lambda data: float(data.replace(",", ""))
)

In [ ]:
kosdaq_df = kosdaq_df.drop(columns=["closePrice"])
usd_df = usd_df.drop(columns=["closePrice"])

In [ ]:
merge_df_1 = pd.merge(kospi_df, kosdaq_df, on="localTradedAt")
merge_df_2 = pd.merge(merge_df_1, usd_df, on="localTradedAt")
merge_df = merge_df_2.copy()
merge_df.tail(2)

In [ ]:
# 시각화
plt.figure(figsize=(20, 5))

# plt.plot(merge_df["localTradedAt"], merge_df["kospi"], label="kospi")
# plt.plot(merge_df["localTradedAt"], merge_df["kosdaq"], label="kosdaq")
# plt.plot(merge_df["localTradedAt"], merge_df["usd"], label="usd")

columns = merge_df.columns[1:]
for column in columns:
    plt.plot(merge_df["localTradedAt"], merge_df[column], label=column)
    
xticks_count = 11
plt.xticks(
    merge_df["localTradedAt"][::int(len(merge_df) // xticks_count) + 1]
)
plt.legend(loc=0)
plt.show()

#### 7. 데이터 스케일링
- min max scaling


- $z = \frac{x_i - min(x)}{max(x) - min(x)} (0 \leqq z \leqq 1)$


- latex syntax : `https://jjycjnmath.tistory.com/117`

In [ ]:
from sklearn.preprocessing import minmax_scale

In [ ]:
# 시각화
plt.figure(figsize=(20, 5))

columns = merge_df.columns[1:]
for column in columns:
    plt.plot(
        merge_df["localTradedAt"], 
        minmax_scale(merge_df[column]), 
        label=column
    )
    
xticks_count = 11
plt.xticks(
    merge_df["localTradedAt"][::int(len(merge_df) // xticks_count) + 1]
)
plt.legend(loc=0)
plt.show()

#### 8. 상관관계 분석
- 피어슨 상관계수(Pearson Correlation Coefficient)
- 두 데이터 집합의 상관도를 분석할때 사용되는 지표
- 상관계수의 해석
    - -1에 가까울수록 서로 반대방향으로 움직임
    - 1에 가까울수록 서로 같은방향으로 움직임
    - 0에 가까울수록 두 데이터는 관계가 없음

In [ ]:
# 해석 1 : kospi, kosdaq은 아주 강한 양의 상관관계를 갖는다. (데이터가 같은 방향으로 움직임)
# 해석 2 : kospi와 usd를 약한 양의 상관관계를 갖는다.
corr_df = merge_df[merge_df.columns[1:]].corr()
corr_df

In [ ]:
# 결정계수 : r-squared 
# 1과 가까울수록 강한 관계, 0과 가까울수록 약한 관계
plt.figure(figsize=(20, 5))
sns.heatmap(corr_df ** 2, cmap='YlGnBu', annot=True)
plt.show()